In [1]:
%matplotlib inline
#%tensorflow_version 2.x

import numpy as np
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import time

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
import random
#from google.colab import files
import io

%matplotlib inline

np.random.seed(0)
print("OK")
import tensorflow as tf
print("Tensorflow version", tf.__version__)

OK
Tensorflow version 2.3.0


In [2]:
train_df = pd.read_csv('input/train.csv', index_col='Id')
col_names = train_df.columns.tolist()

print("Column names:")
print(col_names)

print("\nSample data:")
train_df.head()

Column names:
['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']

Sample data:


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Data Preparation

In [3]:
# Isolate target data
cover_type = train_df['Cover_Type']
Y = np.array(cover_type)

# Dropping output from feature space
to_drop = ['Cover_Type']
cover_feat_space = train_df.drop(to_drop, axis = 1)

# Pull out features for future use
features = cover_feat_space.columns
X = np.array(cover_feat_space).astype(np.float)
    
print("Feature space hold %d observations and %d features" % X.shape)
print("Unique target labels:", np.unique(Y))

Feature space hold 15120 observations and 54 features
Unique target labels: [1 2 3 4 5 6 7]


In [4]:
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]
numExamples = int(0.7*X.shape[0])
#test_data, test_labels = X[15120-numExamples:], Y[15120-numExamples:]
train_data, train_labels = X[:numExamples], Y[:numExamples]
test_data, test_labels = X[numExamples:], Y[numExamples:]
numFeatures = train_data[1].size
numTrainExamples = train_data.shape[0]
numTestExamples = test_data.shape[0]
print('Features = %d' %(numFeatures))
print('Train set = %d' %(numTrainExamples))
print('Test set = %d' %(numTestExamples))

Features = 54
Train set = 10584
Test set = 4536


In [5]:
train_labels_b = label_binarize(train_labels, classes=[1,2,3,4,5,6,7])
test_labels_b = label_binarize(test_labels, classes=[1,2,3,4,5,6,7])
numClasses = train_labels_b[1].size
print('Classes = %d' %(numClasses))

Classes = 7


In [6]:
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

### Baseline

In [7]:
neighbors = 1
knn = KNeighborsClassifier(neighbors)
# we'll be waiting quite a while if we use 60K examples, so let's cut it down.  You may want to run the full 60K on your own later to see what the accuracy is.
#mini_train_data, mini_train_labels = X[:numExamples], Y[:numExamples] 
start_time = time.time()
knn.fit(train_data, train_labels)
print ('Train time = %.2f' %(time.time() - start_time))
start_time = time.time()
accuracy = knn.score(test_data, test_labels)
print ('Accuracy = %.4f' %(accuracy))
print ('Prediction time = %.2f' %(time.time() - start_time))

Train time = 0.19
Accuracy = 0.8003
Prediction time = 2.65


### Neural Nets

In [8]:
## Model
model = Sequential() 
model.add(Dense(7, input_dim=54, activation='softmax')) 
# How many weights will we have...?

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
start_time = time.time()
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=numTrainExamples, verbose=0, epochs=50) 
print ('Train time = %.2f' %(time.time() - start_time))
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Train time = 0.51
Test score: 2.0345675945281982
Test accuracy: 0.28703704476356506


In [9]:
## Model
model = Sequential() 
model.add(Dense(7, input_dim=54, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
start_time = time.time()
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=1,verbose=0, epochs=50) 
print ('Train time = %.2f' %(time.time() - start_time))
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Train time = 182.62
Test score: 0.7647470235824585
Test accuracy: 0.6823192238807678


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 7)                 385       
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.evaluate(test_data, test_labels_b, verbose=0) 

[0.7647470235824585, 0.6823192238807678]

### Hidden layers

In [12]:
## Model
model = Sequential() 
model.add(Dense(units=50, input_dim=54, activation='sigmoid')) 
model.add(Dense(units=7, input_dim=50, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=100) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.6093182563781738
Test accuracy: 0.7466931343078613


In [13]:
## Model
model = Sequential() 
model.add(Dense(units=30, input_dim=54, activation='sigmoid')) 
model.add(Dense(units=7, input_dim=30, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=100) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.612201988697052
Test accuracy: 0.7458112835884094


In [14]:
## Model - New activation
model = Sequential() 
model.add(Dense(units=50, input_dim=54, activation='relu')) 
model.add(Dense(units=7, input_dim=50, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=100) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.5483254790306091
Test accuracy: 0.7861552238464355


In [15]:
## Model
model = Sequential() 
model.add(Dense(units=50, input_dim=54, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(units=7, input_dim=50, activation='softmax')) 

## Cost function & Objective (and solver)
sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_data, train_labels_b, shuffle=False, batch_size=10,verbose=0, epochs=120) 
score = model.evaluate(test_data, test_labels_b, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.5622333884239197
Test accuracy: 0.7654321193695068
